# Benchmark code inter-processo

In questo notebook si vogliono evidenziare prestazioni e possibili criticità
della coda utilizzata per la comunicazione interprocesso, ossia
`multiprocessing.Queue`.

La coda permette di implementare facilmente un paradigma _produttore-consumatore_
largamente usato per distribuire task a dei processi _worker_.

Essendo la comunicazione inter-processo, ogni oggetto inserito nella coda viene
serializzato e viene deserializzato nel momento in cui viene estratto. Il modulo
usato per serializzare i vari oggetti è `pickle`.

Il benchmark è molto semplice: verranno inseriti e rimossi vari oggetti, di
diverse dimensioni nella coda. Lo stesso oggetto verrà inserito (e rimosso) 100
volte al fine di avere un dato statistico solido. Saranno poi calcolati tempo
medio e deviazione standard dei tempi di inserimento e rimozione dalla coda.

Come oggetto da inviare utilizzeremo degli array `numpy`, i quali hanno una
dimensione ben definita, sia in termini di struttura, sia in termini di
attributi.

In [1]:
import numpy as np

a = np.array([i for i in range(10)])
print(f"numpy array size: {a.nbytes} bytes")

numpy array size: 80 bytes


Come ci si poteva aspettare, `numpy` inizializza vettori di valori a 8 byte
a meno che non si specifichi un tipo diverso.

Definiamo quindi la funzione che testa le prestazioni di inserimento e rimozione
in questo modo:

In [5]:
import multiprocessing as mp
import time

def benchmark(dim: int):
    q = mp.Queue()
    buffer = np.array([i for i in range(dim)])
    
    put_times = []
    get_times = []
    for _ in range(1000):
        start = time.perf_counter()
        q.put(buffer)
        end_put = time.perf_counter()
        q.get()
        end = time.perf_counter()

        put_times.append(end_put - start)
        get_times.append(end - end_put)

    return np.array(put_times), np.array(get_times)

dims = np.around(np.geomspace(100, 51200, 10)).astype(int)
dims

array([  100,   200,   400,   800,  1600,  3200,  6400, 12800, 25600,
       51200])

In [4]:
for d in dims:
    put_times, get_times = benchmark(d)
    print(put_times.mean(), get_times.mean())

7.272063980053645e-06 6.907236398910754e-05
5.9001990048273e-06 6.207102800908615e-05
2.6670589904824737e-06 2.6598221022140933e-05
7.3679580345924475e-06 7.865222404507221e-05
5.352020085410913e-06 6.396594295438263e-05
5.722862006223295e-06 8.515183897179668e-05
4.372673032776219e-06 7.753501095066894e-05
2.61379206131096e-06 6.063484695914667e-05
2.873162073228741e-06 0.00011005493501579622
3.2325740030501037e-06 0.0003620079239772167
